In [8]:
''' 
implementation of the belief algorithm
parameters: 
- kc: size of the cache
- v: vote size 
- h: length of the time window
- kt

'''
history_ex = [1,2,6,5,4,2,5,1,4,8,3,6,2,5,4,5,1,7,2,6,4,5,4,1,4,2,1,3,6,5,4,8,5,4,2,9,6,3,5,2,1,2,1,2,4,5,6,1,2,7,8,4]
size_cache_ex = 4



In [19]:
# comments: I had to guess the definition of two things that were not formally defined in the paper
# this function can be implement in a way that we don't recalculate everything for each current element

def calcul_beliefs(history, h: int): 
    # we want to calculate the belief (x,o) that page x will be accessed during a time window of size h after after page o; 
    # to calculate this belief we use the previous pages requested history
    out = dict()  # the function outputs a dictionnary indexed by (x,o) 
    popularity = dict() # we also need to count the 'popularity' of each x, ie the number of lenght-h windows in which x appear. UWAGA: it is not precisely defined in the paper, I had to guess
    for i in range(len(history)-h): # in the paper, they only calculate the belief for full windows; I wonder if it is the right way to do: we could also consider calculating beliefs using partial window (of size less that h) at the end of the history
        o = history[i]
            
        pages_to_appear_in_the_window = [] 
        for j in range(1,h+1):
            x = history[i+j]
            #updating counts for belief:
            if (x,o) in out.keys(): 
                out[(x,o)]+=1
            else:
                out[(x,o)]=1
            if x not in pages_to_appear_in_the_window:
                pages_to_appear_in_the_window.append(x)
        
        # updating popularity:
        for x in pages_to_appear_in_the_window:
            if x in popularity.keys(): 
                popularity[x]+=1
            else:
                popularity[x]=1

    # calculating the frequencies:
    nb_occurrences = dict() # to compute the belief, we also need to count the number of occurences of the pages. UWAGA: it is not precisely defined in the paper, I had to guess
    for x in history:
        if x in nb_occurrences.keys(): 
            nb_occurrences[x]+=1
        else:
            nb_occurrences[x]=1
    
    # final calculation of the beliefs: 
    for (x,o) in out.keys():
        out[(x,o)]=out[(x,o)]*nb_occurrences[x]/(nb_occurrences[o]*popularity[x])
    
    return out
    
    

In [32]:
# example:
calcul_beliefs(history_ex,4)

{(2, 1): 0.26515151515151514,
 (6, 1): 0.17045454545454544,
 (5, 1): 0.10887096774193548,
 (4, 1): 0.3125,
 (6, 2): 0.13636363636363635,
 (5, 2): 0.26129032258064516,
 (4, 2): 0.21875,
 (2, 2): 0.12121212121212122,
 (5, 6): 0.3870967741935484,
 (4, 6): 0.2604166666666667,
 (2, 6): 0.20202020202020202,
 (4, 5): 0.2777777777777778,
 (2, 5): 0.2356902356902357,
 (5, 5): 0.0967741935483871,
 (1, 5): 0.28444444444444444,
 (2, 4): 0.24242424242424243,
 (5, 4): 0.14516129032258066,
 (1, 4): 0.224,
 (4, 4): 0.15625,
 (1, 2): 0.224,
 (8, 2): 0.03,
 (8, 5): 0.06666666666666667,
 (3, 5): 0.027777777777777776,
 (8, 1): 0.075,
 (3, 1): 0.09375,
 (8, 4): 0.06,
 (3, 4): 0.075,
 (6, 4): 0.10909090909090909,
 (3, 8): 0.08333333333333333,
 (6, 8): 0.09090909090909091,
 (2, 8): 0.20202020202020202,
 (5, 8): 0.1935483870967742,
 (6, 3): 0.18181818181818182,
 (2, 3): 0.30303030303030304,
 (5, 3): 0.2903225806451613,
 (4, 3): 0.20833333333333334,
 (7, 5): 0.09523809523809523,
 (7, 4): 0.02857142857142857,
 

In [ ]:
def prefetch_suggest(history, current_cache, h: int, kc: int, v: int):
    '''
    parameters:
    h: length of the time window for the calculation of beliefs values
    kc: number of cache candidates
    v: number of voters (comment: in the paper there is a typo, they seem to mix it with kc; thus, we may imagine that v=kc); v has to be at least one. 
    
    '''
    if len(history) == 0: 
        return []
    o = history[-1] # the current page o (history has to contain at least one page)
    if o not in history[:-(h+1)]: # if o has been requested before #UWAGA: not clear in the paper what is the right condition
        return []
    beliefs = calcul_beliefs(history, h) # we calculate the beliefs (in fact we just need those with index (x,o) for all x != o); comment: why not also x=o ?? this is not precised in the paper
    # we create a list of kc cache candidates, that are not in the cache, and that have maximum belief (with respect to the current element o)
    # comment: there are probably some smart functions to do that but I don't kwon, so I just re-implement it. UWAGA: this part can be improved later
    cache_candidates = []
    for x in history: 
        if x != o and (x,o) in beliefs.keys() and (x not in current_cache) and (x not in cache_candidates): 
            cache_candidates.append(x)
        cache_candidates.sort(reverse=True, key = lambda x : beliefs[(x,o)])
        cache_candidates = cache_candidates[:kc] # we keep at most kc candidates
    
    # now we calculate the maximum belief among v last access elements
    maximum_belief = dict() # maximum_belief[x] corresponds to x_max in the paper (line 12)
    for x in cache_candidates + current_cache:
        maximum_belief[x]=beliefs[(x,o)]
        for oo in history[-v:-1]: # for all voters, ie the last v accessed elements (in the history), except the current element that we already took into account 
            maximum_belief[x] = max(maximum_belief[x], beliefs[(x,oo)])
    
    sorted_list = cache_candidates + current_cache # we create a 'sorted list' as in line 14
    sorted_list.sort(reverse=True, key = lambda x : maximum_belief[x]) # and we sort this list by decreasing maximum beliefs
    
    
    
    

In [38]:
# test area
o = history_ex[-1]
beliefs=calcul_beliefs(history_ex,4)
cache_candidates = []
for x in history_ex: 
        if x != o and (x,o) in beliefs.keys() and (x not in [5,1]) and (x not in cache_candidates): 
            cache_candidates.append(x)
        cache_candidates.sort(reverse=True, key = lambda x : beliefs[(x,o)])
        cache_candidates = cache_candidates
cache_candidates

[2, 6, 3]

In [40]:
[5,1,9,6,2,4,5,1,4][-3:-1]

[5, 1]

In [27]:
(lambda x: x+x)(1)

2